In [1]:
import pandas as pd

In [5]:
#files
template = pd.read_excel('template.xlsx', sheet_name="2025-Tasks")
df = pd.read_csv('input.csv')

In [11]:
df_keys = df.keys()
template_keys = template.keys()

print(f"keys of the template file: {template_keys}")
print(f"keys of the input file: {df_keys}")

keys of the template file: Index(['Module / Category', 'Task Details', 'Assigned To', 'Start Date',
       'Due Date', 'Planned (Hrs)', 'Actual (Hrs)', 'Status', 'End Date',
       'Risk / Comments / Comp Off'],
      dtype='object')
keys of the input file: Index(['Issue Type', 'Issue key', 'Issue id', 'Summary', 'Assignee',
       'Assignee Id', 'Reporter Id', 'Priority', 'Status', 'Resolution',
       'Custom field (Start date)', 'Due date', 'Σ Original Estimate',
       'Σ Time Spent'],
      dtype='object')


In [ ]:
template['Task Details'] = df['Summary']
template['Assigned To'] = df['Assignee']
template['Start Date'] = pd.to_datetime(df['Custom field (Start date)'], dayfirst=True).dt.strftime('%d-%b-%y')
template['Due Date'] = pd.to_datetime(df['Due date'], dayfirst=True).dt.strftime('%d-%b-%y')
template['Planned (Hrs)'] = df['Σ Original Estimate'] / 3600
template['Actual (Hrs)'] = df['Σ Time Spent'] / 3600
template['Status'] = df['Status'].replace('Done', 'Completed')

template.to_excel('artifacts/unnecessary/output.xlsx', index=False)

In [ ]:
import pandas as pd
from openpyxl import load_workbook

try:
    # Read CSV
    df = pd.read_csv('input.csv')
    print(f"CSV loaded: {df.shape}")
    
    # Load workbook
    workbook = load_workbook('template.xlsx')
    worksheet = workbook['2025-Tasks']
    
    # Start from row 2 (assuming row 1 has headers)
    start_row = 2
    
    print(f"Writing data starting from row: {start_row}")
    
    # Process data
    for i, row in df.iterrows():
        current_row = start_row + i
        
        if i < 3:
            print(f"Writing to row {current_row}: {row['Summary']}")
        
        worksheet.cell(row=current_row, column=2, value=str(row['Summary']))  # Changed from column=1 to column=2
        worksheet.cell(row=current_row, column=3, value=str(row['Assignee']))  # Changed from column=2 to column=3
        
        # Format dates
        try:
            start_date = pd.to_datetime(row['Custom field (Start date)'], dayfirst=True).strftime('%d-%b-%y')
            due_date = pd.to_datetime(row['Due date'], dayfirst=True).strftime('%d-%b-%y')
        except:
            start_date = str(row['Custom field (Start date)'])
            due_date = str(row['Due date'])
        
        worksheet.cell(row=current_row, column=4, value=start_date)  # Changed from column=3 to column=4
        worksheet.cell(row=current_row, column=5, value=due_date)    # Changed from column=4 to column=5
        worksheet.cell(row=current_row, column=6, value=float(row['Σ Original Estimate']) / 3600 if pd.notna(row['Σ Original Estimate']) else 0)  # Changed from column=5 to column=6
        worksheet.cell(row=current_row, column=7, value=float(row['Σ Time Spent']) / 3600 if pd.notna(row['Σ Time Spent']) else 0)  # Changed from column=6 to column=7
        
        status = 'Completed' if row['Status'] == 'Done' else str(row['Status'])
        worksheet.cell(row=current_row, column=8, value=status)  # Changed from column=7 to column=8
    
    # Save file
    workbook.save('template.xlsx')
    print(f"Data saved to template_with_data.xlsx starting from row {start_row}")
    workbook.close()
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle

def process_csv_to_excel(input_csv_path):
    """
    Process CSV data and write to Excel template if 'Parent summary' column is not present.
    
    Args:
        input_csv_path (str): Path to the input CSV file
    """
    try:
        # Read CSV
        df = pd.read_csv(input_csv_path)
        print(f"CSV loaded: {df.shape}")
        
        # Check if 'Parent summary' column exists
        if 'Parent summary' in df.columns:
            print("'Parent summary' column found in CSV. Skipping processing.")
            return
        
        print("'Parent summary' column not found. Proceeding with processing...")
        
        # Load workbook
        workbook = load_workbook('artifacts/template.xlsx')
        worksheet = workbook['2025-Tasks']
        
        # Start from row 2 (assuming row 1 has headers)
        start_row = 2
        
        print(f"Writing data starting from row: {start_row}")
        
        # Process data
        for i, row in df.iterrows():
            current_row = start_row + i
            
            if i < 3:
                print(f"Writing to row {current_row}: {row['Summary']}")
            
            worksheet.cell(row=current_row, column=2, value=str(row['Summary']))
            worksheet.cell(row=current_row, column=3, value=str(row['Assignee']))
            
            # Format dates
            try:
                start_date = pd.to_datetime(row['Custom field (Start date)'], dayfirst=True).strftime('%d-%b-%y')
                due_date = pd.to_datetime(row['Due date'], dayfirst=True).strftime('%d-%b-%y')
            except:
                start_date = str(row['Custom field (Start date)'])
                due_date = str(row['Due date'])
            
            worksheet.cell(row=current_row, column=4, value=start_date)
            worksheet.cell(row=current_row, column=5, value=due_date)
            
            # Set hours values with number formatting
            planned_hours = float(row['Σ Original Estimate']) / 3600 if pd.notna(row['Σ Original Estimate']) else 0
            actual_hours = float(row['Σ Time Spent']) / 3600 if pd.notna(row['Σ Time Spent']) else 0
            
            planned_cell = worksheet.cell(row=current_row, column=6, value=planned_hours)
            actual_cell = worksheet.cell(row=current_row, column=7, value=actual_hours)
            
            # Set number format to prevent percentage display
            planned_cell.number_format = '0.00'
            actual_cell.number_format = '0.00'
            
            status = 'Completed' if row['Status'] == 'Done' else str(row['Status'])
            worksheet.cell(row=current_row, column=8, value=status)
        
        # Save file
        workbook.save('template.xlsx')
        print(f"Data saved to template.xlsx starting from row {start_row}")
        workbook.close()
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

In [2]:
# Usage example:
# process_csv_to_excel('input.csv')
process_csv_to_excel('artifacts/unnecessary/input.csv')

CSV loaded: (253, 14)
'Parent summary' column not found. Proceeding with processing...
Writing data starting from row: 2
Writing to row 2: Monthly Team Meeting
Writing to row 3: Email Engagements to Hot deals
Writing to row 4: BFS Strategy meeting
Data saved to template.xlsx starting from row 2
